In [ ]:
# input data (fichier .xmind)
input_path = r'C:\Users\jl0331el\OneDrive - EDF\XMind\MYPLANTONLINE CETAC.xmind'

# output data (fichier .xlsx)
output_excel_path = r"C:\Users\jl0331el\OneDrive - EDF\XMind\output_excel\exported_data1.xlsx"

In [2]:
import xmindparser
import logging
import json
import pprint

# Configurer le parsing avant utiliser xmind_to_dict
# parce qu'il se peut que la config dans le main.py du package soit pas la bonne
class XmindToExcel:
    
    def __init__(self):
        # paramétrer le logging
        logging.basicConfig(level=logging.INFO)
        

    def config(self, show_topic_id=True, hide_empty_value=False):
        """
        paramétrer xmindparser
        :param show_topic_id: je veux montrer le topic id
        :param hide_empty_value: je ne veux pas cacher les valeur vides
        """
        xmindparser.config['showTopicId'] = show_topic_id
        xmindparser.config['hideEmptyValue'] = hide_empty_value
        logging.info(f"Configured: showTopicId={show_topic_id}, hideEmptyValue={hide_empty_value}")
        

    def parsing(self, xmind_file, output_format='dict'):

        try:
            # Parser le fichier .xmind
            parsed_data = xmindparser.xmind_to_dict(xmind_file)

            # retourner la données sous format json ou dict (préférée)
            if output_format == 'json':
                return json.dumps(parsed_data, indent=4, ensure_ascii=False)
            elif output_format == 'dict':
                return parsed_data
            else:
                raise ValueError("Invalid output format. Use 'dict' or 'json'.")
        except Exception as e:
            logging.error(f"Error parsing the XMind file: {e}")
            raise

# Initialiser la configuration
if __name__ == "__main__":
    xmind_excel = XmindToExcel()
    xmind_excel.config(show_topic_id=True, hide_empty_value=False)

# Example usage
    try:
        parsed_result = xmind_excel.parsing(input_path, output_format='dict')
        pprint.pprint(parsed_result)  # Prints output
    except Exception as e:
        logging.error(f"Failed to parse the XMind file: {e}")

INFO:root:Configured: showTopicId=True, hideEmptyValue=False


[{'id': '5tbqghjog54v7fqs5fo2mtvvdq',
  'structure': 'org.xmind.ui.map.unbalanced',
  'title': 'Feuille  1',
  'topic': {'comment': None,
            'id': '6blfuqug2s86jmr5dggbgn8e7b',
            'labels': None,
            'link': None,
            'makers': None,
            'note': None,
            'title': 'SYNOPTIQUES ',
            'topics': [{'comment': None,
                        'id': '6erlacb0fd3pfek05i9j8g14tg',
                        'labels': None,
                        'link': 'https://prod-thaf-vision.edf.fr/PIVision/#/Displays/1754',
                        'makers': None,
                        'note': None,
                        'title': 'prévision de fonctionnement'},
                       {'comment': None,
                        'id': '66mvug34huechkgg8fnlp3vfq4',
                        'labels': None,
                        'link': 'https://prod-thaf-vision.edf.fr/PIVision/#/Displays/1685',
                        'makers': None,
                    

In [3]:
# exporter en excel en faisant apparaitre l'hierarchie

import pandas as pd

data= parsed_result

def extract_hierarchy(topics, path=None):
    """
    faire une extraction hierarchique de manière recursivement des données des topics.
    
    Args:
        topics (liste): Liste de topics.
        path (liste): une liste qui maintien l'hierarchie de topics.
    
    Returns:
        une liste de lignes dans lesquelles chaque ligne contient les titres hierarchique et un link
        
    """
    if path is None:
        path = []

    rows = []
    for topic in topics:
        current_path = path + [topic.get('title', '')]  # mettre à jour le path avec le titre en cours
        if 'link' in topic:  # ajouter une ligne avec son link si le link exist
            rows.append(current_path + [topic['link']])
        if 'topics' in topic:  # faire une recurse dans subtopics
            rows.extend(extract_hierarchy(topic['topics'], current_path))
    return rows

# Trouver le topic de base dans mon structure de data
# root_topic = [{'title': data[0]['topic']['title'], 'link': data[0]['topic'].get('link', ''), 'topics': data[0]['topic']['topics']}]
root_topic = [{'title': data[0]['topic']['title'], 'link': data[0]['topic']['link'], 'topics': data[0]['topic']['topics']}]

# extraire lignes dans l'hierarchie
rows = extract_hierarchy(root_topic)

# Determiner le profondeur maximal pour les columns de titre (title levels)
max_depth = max(len(row) - 1 for row in rows)

# Normalize rows to have equal columns (titles and link)
normalized_rows = [row[:-1] + [''] * (max_depth - len(row) + 1) + [row[-1]] for row in rows]

# Create a DataFrame with titles in separate columns and link in the last column
columns = [f'Title Level {i+1}' for i in range(max_depth)] + ['Link']
df = pd.DataFrame(normalized_rows, columns=columns)

# Export to Excel
excel_file_path = output_excel_path
df.to_excel(excel_file_path, index=False)

print(f"Excel file saved to: {excel_file_path}")

Excel file saved to: C:\Users\jl0331el\OneDrive - EDF\XMind\output_excel\exported_data1.xlsx
